### 第5章 AIエージェント×ワークフローによる作業自動化

ライブラリのimport

In [3]:
from openai import OpenAI
import yaml
import json

【注意】下記実行前にREADME.mdに従いルートフォルダにconfig.yamlを作成してください。

In [4]:
with open('config.yaml', 'r') as yml:
    config = yaml.safe_load(yml)

クライアントの作成

In [5]:
client = OpenAI(
    api_key = config["oai"]["key"], # 取得したAPIキー
    # base_url= <URL> # Azure OpenAI Serviceを使う場合は必要
)

5.1 指示プロンプト例

In [6]:
instruction = """

# Role
あなたはコンサルティングファームの週次ニュース調査・事例突合・レポート配信を行うツール駆動エージェントです。
<operating_mode>
- すべての作業はツール呼び出しで完結する。
- 最終出力は必ず send_email の tool call で終了する。
</operating_mode>

# Task
<workflow>
Phase1: 入力理解・期間確定
Phase2: web_search でニュース収集(複数クエリ)
Phase3: 重要度選別・要約・タグ付け
Phase4: case_search で社内事例突合
Phase5: report_request_approval でレポート生成・承認依頼
Phase6: report_request_approval → approved 確認(承認されなければ指摘を反映し再依頼)
Phase7: send_email で配信(最終)
</workflow>

# Input
<input_schema>
(【略】InputのJSONスキーマとそれぞれのプロパティ説明)
</input_schema>

# Tool
Taskのworkflow内におけるPhaseを思考中に必ず確認し、適切なツールを呼び出して作業を進める。

<tool_usage_rules>
- 根拠になった情報は検索結果および格納先のURLを[<リンク>]として必ず付与する。
</tool_usage_rules>


## report_request_approval
出力にはreport_template_markdownテンプレートを用いる
<report_template_markdown>
## サマリ(3〜6行)
- (最重要トピックを最大3点)

## 今週の重要動向(最大5件)
- [重要度: 高/中/低] 見出し

## 過去事例との突合(最大5件)
- 事例ID/パス: ...

## リスク/機会と推奨アクション(最大5件)
- (アクションは「誰が/何を/いつまでに」が分かる書き方)

## 前提・不確実性(必須)
- (推測は推測と明記。未確認は未確認と明記)
</report_template_markdown>

# Policy
<quality_and_grounding>
- 根拠のない断定は禁止。日付・数字・固有名詞は特に厳密。
- 矛盾があれば追加検索し、解消できなければ不一致として記載。
</quality_and_grounding>

<privacy_and_handling>
- 社内事例DBの詳細を社外共有しない前提で、メール本文は要約＋参照ID/パスに留める。
</privacy_and_handling>
"""

report_request_approval_description = """
Purpose: 収集済みニュースと社内事例をもとにレポートMarkdownを生成し、承認依頼として保存・申請します。

Use when:
- Phase5、および承認が下りなかった場合の再承認依頼時
- 対象顧客・期間・想定読者などの前提と必要情報（ニュース要約・事例・推奨アクション）が揃っている

Do not use when:
- Phase5以外
- 収集・分析が未完了で、まず要件や前提条件の確認が必要な状態

Notes:
- 失敗時: 生成したレポートは保持したまま簡潔に指摘内容を反映し、再試行。
"""

report_markdown_description = """
以下の点を遵守してレポートを作成してください。
- レポートはreport_template_markdownテンプレートに従い、冗長な叙述を避ける(箇条書き中心)。
- 入力スコープ(顧客・業界・地域・論点・期間)を逸脱しない。
- 不足情報は「仮定」として明示。
"""


5.2 report_request_approvalツール定義例

In [7]:
tools = [
    {"type": "web_search"},
    {
        "type": "function",
        "name": "report_request_approval",
        "description": report_request_approval_description,
        "strict": True,
        "parameters": {
            "type": "object",
            "properties": {
            "report_markdown": {
                "type": "string",
                "description": report_markdown_description
            }
            },
            "required": ["report_markdown"],
            "additionalProperties": False
        }
    }
]

5.5 リクエスト例

In [8]:
messages=[
        {"role": "developer", "content": instruction},
        {"role": "user", "content": "鉄道業界の架空のレポートが出せる状態になった想定で、Phase5に入ったとして、ツール選択してください。レポートは架空のものを作ってください。"}
    ]

response = client.responses.create(
    instructions=instruction,
    model="gpt-5-mini",
    input=messages,
    tools=tools,
    tool_choice="required"
)

response_type = response.output[1].type
function_name = response.output[1].name
arguments = json.loads(response.output[1].arguments)
id = response.output[1].call_id

print("response_type: ", response_type)
print("function_name: ", function_name)
print("argument: ", arguments)
print("id: ", id)

response_type:  function_call
function_name:  report_request_approval
argument:  {'report_markdown': '## サマリ(3〜6行)\n- 全国的な地方路線の利用者回復の停滞と、自治体・事業者による近代化補助金の増額が同時進行している（架空）。[参照: 政府補助案概要の抜粋][https://news.example.com/rail-future-jan2026]\n- 予知保全・デジタル双方向プラットフォームへの投資が加速。大手私鉄・第三セクターでパイロット導入が進行中（架空）。[参照: 事例概要][https://internal.example.com/case_summary/pdm2026]\n- 貨物需要の回復に伴い、複合輸送での鉄道比率拡大の機会が出現。短期は設備・運用調整が必要（架空）。[参照: 物流業界調査][https://news.example.com/logistics-rail-jan2026]\n\n## 今週の重要動向(最大5件)\n- [重要度: 高] 政府の『地域鉄道近代化補助プログラム』増額案（公募開始想定：2026-02-15）。公的資金で老朽設備更新とデジタル化を同時支援。[資料: 補助案概要][https://news.example.com/rail-future-jan2026]\n- [重要度: 高] 大手私鉄A社が予知保全プラットフォームを本格運用へ（パイロット完了、段階的展開2026 Q2開始想定）。保守コスト削減と稼働率改善を見込む。[社内メモ][https://internal.example.com/case_summary/pdm2026]\n- [重要度: 中] 都市部通勤需要はコロナ前比で85〜92%のレンジで停滞。テレワーク定着が継続的影響を与える見通し（短期需要リスク）。[市場データ][https://news.example.com/ridership-data-jan2026]\n- [重要度: 中] 物流業界で鉄道回帰の動き。長距離トラックコスト上昇により、鉄道貨物の競争力が改善（機会）。[業界レポート][https://news.example.com/logistics-rail-jan20

5.7 Conversations APIの履歴管理

In [9]:
# 会話履歴の格納先を生成
conversation = client.conversations.create()

# ツール使用結果とReasoning過程を会話履歴に含める設定でLLMにテキスト生成リクエスト
response = client.responses.create(
  model="gpt-5-nano",
  input=[
    {"role": "system", "content": "要求された内容だけ答えてください。"},
    {"role": "user", "content": "LLMのRAGに関連するキーワードとその簡易解説を1つ出力して。"}
  ],
  tools = [
    {"type": "web_search"}
  ],
  reasoning = {"summary":"detailed"},
  tool_choice="required",
  include=["web_search_call.action.sources",  "reasoning.encrypted_content"],
  conversation=conversation.id
)

5.8 ConversationsAPIの履歴出力例

In [10]:
for resp in response.output:
    # print(resp)
    if resp.type == 'reasoning':
        print(json.dumps({"type": resp.type, "summary": [s.text for s in resp.summary]}, ensure_ascii=False, indent=2))
    elif resp.type == 'web_search_call':
        print(json.dumps({"type": resp.type, "query": resp.action.query}, ensure_ascii=False, indent=2))
    elif resp.type == 'message':
        print(json.dumps({"type": resp.type, "role": resp.role, "text": [c.text for c in resp.content if hasattr(c, 'text')]}, ensure_ascii=False, indent=2))

{
  "type": "reasoning",
  "summary": [
    "**Considering web.run call**\n\nI need to make sure to call one of the defined tools immediately in response to the user's request, so I’m planning to perform a web.run call. I’ll provide a keyword like \"RAG: Retrieval-Augmented Generation\" and a simple explanation. I need to verify this through a search query to ensure accuracy. Since I'll be using web.run, I’ll also need to include citations for the information I gather. It’s essential to keep this straightforward while being accurate!",
    "**Planning citations for RAG**\n\nThe instructions say that if I call web.run, all statements based on internet sources should have citations. I’m thinking of using \"Keyword: Retrieval-Augmented Generation (RAG)\" with an explanation that LLMs fetch relevant documents and generate answers. It's essential to reference authoritative sources, like the Facebook AI paper or possibly Wikipedia. I’ll need to perform web.run with four queries about RAG to 

5.9 ConversationsAPIの履歴管理でReasoning過程を除外して再利用

In [11]:
items = client.conversations.items.list(conversation.id, limit=10)

for item in items.data:
    # print(item)
    if item.type == 'reasoning':
        print(json.dumps({"type": item.type, "summary": [s.text for s in item.summary] if hasattr(item, 'summary') else []}, ensure_ascii=False, indent=2))
    elif item.type == 'web_search_call':
        sources = getattr(item.action, "sources", None) if hasattr(item, 'action') else None
        print(json.dumps({
            "type": item.type,
            "urls": [src.url for src in sources] if sources else [],
            "query": getattr(item.action, "query", None) if hasattr(item, 'action') else None
        }, ensure_ascii=False, indent=2))
    elif item.type == 'message':
        print(json.dumps({"type": item.type, "role": item.role, "text": [c.text for c in item.content if hasattr(c, 'text')]}, ensure_ascii=False, indent=2))

{
  "type": "message",
  "role": "assistant",
  "text": [
    "キーワード: Retrieval-Augmented Generation（RAG）\n\n簡易解説: LLMが外部データを検索して取り込み、その retrieved ドキュメントを前提に回答を生成する手法。知識集約タスクの正確性を高め、最新情報の反映や出典の提示を容易にする。出典として原著論文（2020年）と解説ページがある。 ([arxiv.org](https://arxiv.org/abs/2005.11401?utm_source=openai))"
  ]
}
{
  "type": "reasoning",
  "summary": []
}
{
  "type": "web_search_call",
  "urls": [],
  "query": "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks arXiv"
}
{
  "type": "reasoning",
  "summary": []
}
{
  "type": "web_search_call",
  "urls": [],
  "query": "Retrieval-Augmented Generation RAG paper arXiv Lewis 2020"
}
{
  "type": "reasoning",
  "summary": [
    "**Considering web.run call**\n\nI need to make sure to call one of the defined tools immediately in response to the user's request, so I’m planning to perform a web.run call. I’ll provide a keyword like \"RAG: Retrieval-Augmented Generation\" and a simple explanation. I need to verify this through a search query to e

5.9 ConversationsAPIの履歴管理でReasoning過程を除外して再利用

In [12]:
reasoning_msg_ids = [itm.id for itm in items.data if itm.type == "reasoning"]

for msg_id in reasoning_msg_ids:
   new_conversation = client.conversations.items.delete(conversation_id=conversation.id, item_id=msg_id)

items = client.conversations.items.list(new_conversation.id, limit=10)
 
for item in items.data:
    # print(item)
    if item.type == 'reasoning':
        print(json.dumps({"type": item.type, "summary": [s.text for s in item.summary] if hasattr(item, 'summary') else []}, ensure_ascii=False, indent=2))
    elif item.type == 'web_search_call':
        sources = getattr(item.action, "sources", None) if hasattr(item, 'action') else None
        print(json.dumps({
            "type": item.type,
            "urls": [src.url for src in sources] if sources else [],
            "query": getattr(item.action, "query", None) if hasattr(item, 'action') else None
        }, ensure_ascii=False, indent=2))
    elif item.type == 'message':
        print(json.dumps({"type": item.type, "text": [c.text for c in item.content if hasattr(c, 'text')]}, ensure_ascii=False, indent=2))

{
  "type": "message",
  "text": [
    "キーワード: Retrieval-Augmented Generation（RAG）\n\n簡易解説: LLMが外部データを検索して取り込み、その retrieved ドキュメントを前提に回答を生成する手法。知識集約タスクの正確性を高め、最新情報の反映や出典の提示を容易にする。出典として原著論文（2020年）と解説ページがある。 ([arxiv.org](https://arxiv.org/abs/2005.11401?utm_source=openai))"
  ]
}
{
  "type": "web_search_call",
  "urls": [],
  "query": "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks arXiv"
}
{
  "type": "web_search_call",
  "urls": [],
  "query": "Retrieval-Augmented Generation RAG paper arXiv Lewis 2020"
}
{
  "type": "message",
  "text": [
    "LLMのRAGに関連するキーワードとその簡易解説を1つ出力して。"
  ]
}
{
  "type": "message",
  "text": [
    "要求された内容だけ答えてください。"
  ]
}


5.10 Compact機能の使用例

###### ※この例ではサンプルが小さいため、圧縮しても効果が薄いor元よりもトークンが大きくなる点に注意

In [13]:
# ダミーの会話を用意
messages = [
    {"role":"system","content":"あなたはLLMの専門家です。具体情報を交えて短め(最大50字程度)で答えてください。"},
    {"role":"user","content":"私は中学生です。GPT-5.2とGPT-4.1の位置づけを、用途別に教えてください。"},
    {"role":"assistant","content":"GPT-5.2は推論対応の旗艦で、複雑なコーディングやエージェントに強いです。"},
    {"role":"user","content":"低コスト重視で遅延も抑えたいとき、GPT-5系はどう選びますか。"},
    {"role":"assistant","content":"GPT-5 miniは速く安価で定型タスク向きで、さらに軽量ならGPT-5 nanoが候補です。"},
    {"role":"user","content":"長文を大量に扱う場合、推論なしで高文脈の選択肢はありますか。"},
    {"role":"assistant","content":"推論ステップ不要ならGPT-4.1が約1M文脈で有力で、難問はGPT-5系も併用します。"},
    {"role":"user","content":"最新のモデルはGoogleだと何だろう？調べてくれる？"}
]

response = client.responses.create(
    model="gpt-5-nano",
    tools=[{"type": "web_search"}],
    include=["reasoning.encrypted_content", "web_search_call.action.sources"],
    input=messages,
    store=True
)

print("結果:", response.output_text)

結果: 最新はGemini 3、Googleの最新モデル（2025年11月公開）。([blog.google](https://blog.google/products-and-platforms/products/gemini/gemini-3/?utm_source=openai))


In [14]:
# まずは非圧縮版で実行
normal_response = client.responses.create(
    model="gpt-5-nano",
    tools=[{"type": "web_search"}],
    include=["reasoning.encrypted_content", "web_search_call.action.sources"],
    previous_response_id=response.id,
    input=[{"role": "user", "content": "さっきGPT-5.2って何に強いって言ってました？"}],
)
print("非圧縮の結果:", normal_response.output_text)
print("非圧縮のインプットトークン:", normal_response.usage.input_tokens)


非圧縮の結果: 推論対応の旗艦、複雑なコーディングとエージェントが得意。
非圧縮のインプットトークン: 4901


In [16]:
# これまでの user message + output を compact する
compacted = client.responses.compact(
    model="gpt-5-mini",
    previous_response_id=response.id,
)
compacted_messages = compacted.output

# 次のターンの入力を追加
compacted_messages.append({"role": "user", "content": "さっきGPT-5.2って何に強いって言ってました？"})

# 次に圧縮版で実行
compacted_response = client.responses.create(
    model="gpt-5-nano",
    tools=[{"type": "web_search"}],
    include=["reasoning.encrypted_content", "web_search_call.action.sources"],
    input=compacted_messages,
)

print("圧縮後の結果:", compacted_response.output_text)
print("圧縮後のインプットトークン:", compacted_response.usage.input_tokens)

c:\Users\hirosatogamo\.virtualenvs\pipenv-slS7I_su\Lib\site-packages\pydantic\main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `literal['output_text']` - serialized value may not be as expected [field_name='type', input_value='input_text', input_type=str])
  PydanticSerializationUnexpectedValue(Expected `ResponseOutputRefusal` - serialized value may not be as expected [field_name='content', input_value=ResponseOutputText(annota...ut_text', logprobs=None), input_type=ResponseOutputText])
  return self.__pydantic_serializer__.to_python(


圧縮後の結果: 最新情報（2025年12月公開）による GPT-5.2 の強みは以下です。
- 長文・長文脈の理解と長期タスクの推進。 ([openai.com](https://openai.com/index/introducing-gpt-5-2//?utm_source=openai))
- 表計算・プレゼン作成・コード作成など、実務的作業を高効率でこなす。 ([openai.com](https://openai.com/index/introducing-gpt-5-2//?utm_source=openai))
- 画像認識・視覚的タスク対応。 ([openai.com](https://openai.com/index/introducing-gpt-5-2//?utm_source=openai))
- ツール呼び出しを含むエージェント的動作・多段階自動化。 ([openai.com](https://openai.com/index/introducing-gpt-5-2//?utm_source=openai))
- GDPval などのベンチマークで専門家レベルに近い/上回る性能。 ([openai.com](https://openai.com/index/introducing-gpt-5-2//?utm_source=openai))
- 複数モード（Instant/Thinking/Pro）。Thinkingは推論重視、Proは長時間推論・高品質。 ([openai.com](https://openai.com/index/introducing-gpt-5-2//?utm_source=openai))
- GPT-5.2-Codexはコーディング・ソフトウェア開発に特化。 ([openai.com](https://openai.com/index/introducing-gpt-5-2-codex/?utm_source=openai))

必要なら公式ページへの案内もします。
圧縮後のインプットトークン: 10575
